# AI(2A) - Human Robot Interaction Project
# 👩‍⚕️  Nurse Robot

## Introduction
This script implements a Spoke Dialog System (SDS) based on Spoken Natural Language Dialog in the context of an automated home nurse assistant, to be used by people with limited capabilities. It simulates actions such as reaching things, provide meals and inspect patient status. The system acquires a spoken input from the user through Google Speech Recognition API. Then, recognizes the intent of the sentence using Rasa NLU and retrieves and action result that is transmitted to the user through VocalWare Text-To-Speech API. The order of the sentences, and the recognition of the utterances and dependencies is processed by the libraries spaCy and Keras, building a model that returns the adequate actions given a series of inputs. 

![title](images/NurseRobotMindmap.png)

In [1]:
#imports for project

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import logging
import rasa_core
from rasa_core.agent import Agent
from rasa_core.policies.keras_policy import KerasPolicy
from rasa_core.policies.memoization import MemoizationPolicy
from rasa_core.interpreter import RasaNLUInterpreter
from rasa_nlu.model import Metadata, Interpreter
from rasa_core.utils import EndpointConfig
from rasa_core.run import serve_application
from rasa_nlu.model import Trainer
from rasa_nlu.training_data import load_data
from rasa_core import config
from IPython.display import IFrame
import pandas as pd
import ruamel
import urllib.request
from urllib.request import urlopen
from playsound import playsound
import uuid
import os
import spacy
from spacy import displacy
import warnings
warnings.simplefilter('ignore', ruamel.yaml.error.UnsafeLoaderWarning)
warnings.filterwarnings("ignore",category=DeprecationWarning)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' #To ignore Tensorflow AVX AVX2 bonary warning

nlp = spacy.load('en_core_web_sm')


Some of the data used in this project is not shown in this notebook, but in external files. The content of each relevant file is described hereunder.

* nurse_domain.yml: Defines the actions that can be taken by the agent, the intents that are expected, the entities to be saved if received, and some answers to the basic user utterances.
* stories.md: Contains the examples of dialogs and the specification of the entities that could be found on the phrases.
* config_spacy.json: Saves the pipeline to be used by the NLP library, in this case sklearn with spaCy 


In [2]:
# Declare paths
domain_file = './nurse_domain.yml'
model_path = './models/dialogue'
interpreter_path ='./models/nursebot/interpreter'
training_data_file = './data/stories.md'
conf_file = './config_spacy.json'

In [3]:
# Build the interpreter of the elements of dialog for the agent. If it doesn't work on the notebook, try on an external console
#!python -m rasa_nlu.train -c config_spacy.json --data data/data.md -o models --fixed_model_name interpreter --project nursebot --verbose

In [4]:
action_endpoint = EndpointConfig(url="http://localhost:5055/webhook")
interpreter = RasaNLUInterpreter(interpreter_path)

In [5]:
if True:
    agent = Agent(domain_file, 
                  policies = [MemoizationPolicy(), KerasPolicy(max_history=3, epochs=200, batch_size=50)],
                  interpreter=interpreter,
                  action_endpoint=action_endpoint)
    data = agent.load_data(training_data_file)
    agent.train(data)
    
    # Save model to be used later
    agent.persist(model_path)

Processed Story Blocks: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 134.58it/s, # trackers=20]
Processed actions: 2766it [00:03, 890.44it/s, # examples=2686]


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 3, 36)             0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                8832      
_________________________________________________________________
dense (Dense)                (None, 16)                528       
_________________________________________________________________
activation (Activation)      (None, 16)                0         
Total params: 9,360
Trainable params: 9,360
Non-trainable params: 0
_________________________________________________________________
Epoch 1/200
1494/1494 [==============================] - 1s 658us/step - loss: 2.6499 - acc: 0.2985
Epoch 2/200
1494/1494 [==============================] - 0s 87us/step - loss: 2.3284 - acc: 0.4458
Epoch 3/200
1494/1494 [==============================] - 0s 93us/step - l

1494/1494 [==============================] - 0s 85us/step - loss: 0.0910 - acc: 0.9679
Epoch 75/200
1494/1494 [==============================] - 0s 84us/step - loss: 0.0971 - acc: 0.9712
Epoch 76/200
1494/1494 [==============================] - 0s 83us/step - loss: 0.0925 - acc: 0.9679
Epoch 77/200
1494/1494 [==============================] - 0s 85us/step - loss: 0.0828 - acc: 0.9739
Epoch 78/200
1494/1494 [==============================] - 0s 88us/step - loss: 0.0932 - acc: 0.9726
Epoch 79/200
1494/1494 [==============================] - 0s 85us/step - loss: 0.0870 - acc: 0.9759
Epoch 80/200
1494/1494 [==============================] - 0s 85us/step - loss: 0.1003 - acc: 0.9679
Epoch 81/200
1494/1494 [==============================] - 0s 85us/step - loss: 0.0825 - acc: 0.9766
Epoch 82/200
1494/1494 [==============================] - 0s 87us/step - loss: 0.0912 - acc: 0.9746
Epoch 83/200
1494/1494 [==============================] - 0s 85us/step - loss: 0.0813 - acc: 0.9739
Epoch 84/200


1494/1494 [==============================] - 0s 86us/step - loss: 0.0440 - acc: 0.9846
Epoch 156/200
1494/1494 [==============================] - 0s 85us/step - loss: 0.0388 - acc: 0.9859
Epoch 157/200
1494/1494 [==============================] - 0s 87us/step - loss: 0.0391 - acc: 0.9846
Epoch 158/200
1494/1494 [==============================] - 0s 84us/step - loss: 0.0455 - acc: 0.9806
Epoch 159/200
1494/1494 [==============================] - 0s 83us/step - loss: 0.0429 - acc: 0.9846
Epoch 160/200
1494/1494 [==============================] - 0s 87us/step - loss: 0.0386 - acc: 0.9886
Epoch 161/200
1494/1494 [==============================] - 0s 85us/step - loss: 0.0505 - acc: 0.9772
Epoch 162/200
1494/1494 [==============================] - 0s 89us/step - loss: 0.0388 - acc: 0.9833
Epoch 163/200
1494/1494 [==============================] - 0s 89us/step - loss: 0.0508 - acc: 0.9826
Epoch 164/200
1494/1494 [==============================] - 0s 84us/step - loss: 0.0405 - acc: 0.9853
Epoc

In [6]:
# Load saved agent, in case no training is needed.
agent = Agent.load('./models/dialogue', interpreter=interpreter, action_endpoint=action_endpoint)

In [7]:
# Generate tree of interaction of utterances in possible dialogs
treeFile = "images/dialogtree.html"
agent.visualize("data/stories.md",output_file=treeFile, max_history=2)
IFrame(src=treeFile, width=1000, height=600)

Processed Story Blocks: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 700.17it/s, # trackers=1]


AttributeError: 'MultiDiGraph' object has no attribute 'node'

In [8]:
interpreter.parse(u"Pass the remote") # Parsed phrase

{'intent': {'name': 'reach', 'confidence': 0.5320247789348383},
 'entities': [],
 'intent_ranking': [{'name': 'reach', 'confidence': 0.5320247789348383},
  {'name': 'move', 'confidence': 0.20616898613224852},
  {'name': 'call', 'confidence': 0.09413365110275207},
  {'name': 'status', 'confidence': 0.045224927224304066},
  {'name': 'thanks', 'confidence': 0.02502236155468827},
  {'name': 'attention', 'confidence': 0.022514995599464748},
  {'name': 'deny', 'confidence': 0.017290703907630267},
  {'name': 'joke', 'confidence': 0.016806219995764298},
  {'name': 'goodbye', 'confidence': 0.016505901665322512},
  {'name': 'greet', 'confidence': 0.015472350218511139}],
 'text': 'Pass the remote'}

In [9]:
# Interpret a sentence
def parsetxt(sent, deptree = False):
    pp = interpreter.parse(sent) # Parsed phrase
    # Print entities present in the sentence
    print(pd.DataFrame(pp['entities']))
    df = pd.DataFrame(pp['intent_ranking']).style.apply(lambda x: ['background: lightgreen' if x.name == 0 else '' for i in x],axis=1)
    if(deptree):
        doc = nlp(sent)
        displacy.render(doc, style='dep', jupyter=True, options={'distance':90})
    return df

In [10]:
# Interpret a sentence
sent1 = u"Nurse, can you pass me the remote and my book, please"
parsetxt(sent1)

Empty DataFrame
Columns: []
Index: []


,confidence,name
0,0.57531,reach
1,0.0817439,status
2,0.0616961,move
3,0.060571,thanks
4,0.0506463,joke
5,0.0432927,call
6,0.0386081,attention
7,0.0264013,goodbye
8,0.0233729,greet
9,0.0231661,deny


In [11]:
# Render Dependency Tree
doc = nlp(sent1)
displacy.render(doc, style='dep', jupyter=True, options={'distance':90})

TypeError: __init__() got an unexpected keyword argument 'encoding'

In [12]:
# Interprete a sentence
sent2 = "Nurse, help me to get to the bathroom please"
parsetxt(sent2)

Empty DataFrame
Columns: []
Index: []


,confidence,name
0,0.441623,move
1,0.187195,joke
2,0.1205,reach
3,0.0636247,call
4,0.0501024,status
5,0.0343805,thanks
6,0.0289445,attention
7,0.0245948,goodbye
8,0.0214918,greet
9,0.0177871,deny


In [13]:
# Render Dependency Tree
doc = nlp(sent2)
displacy.render(doc, style='dep', jupyter=True, options={'distance':110})

In [14]:
parsetxt("call the doctor and tell me a joke")

   confidence  end  entity extractor  start   value
0    0.724851   15  callto   ner_crf      9  doctor


,confidence,name
0,0.243571,joke
1,0.240259,reach
2,0.185137,move
3,0.16123,call
4,0.0411208,thanks
5,0.0398082,status
6,0.0217837,deny
7,0.0201348,attention
8,0.0186381,greet
9,0.0184519,goodbye


In [13]:
#Script to get the Text-To-Speech output.
audioFolder = "./audios/"
effect = ["",
          "&FX_TYPE=P&FX_LEVEL=2", #Pitch
          "&FX_TYPE=D&FX_LEVEL=-1" #Duration
         ]
rooturl = ["https://cache-a.oddcast.com/tts/gen.php?EID=3&LID=1&VID=3&TXT=",
           "&IS_UTF8=1&ACC=3314795&API=2292376&CB=vw_mc.vwCallback&HTTP_ERR=1&vwApiVersion=2"]
characters ={"," : "%2C"," " : "%20","?" : "%3F","'" : "%27"}
def play(phrase, ef = 0):
    for ch in characters:
        phrase = phrase.replace(ch,characters[ch]) #Replace character to match type in URL
    url = rooturl[0] + phrase + effect[ef]+ rooturl[1] #Build the URL
    file = audioFolder + str(uuid.uuid4()) +'.mp3'
    urllib.request.urlretrieve(url, file)
    playsound(file, True)
    #print(file)
    os.remove(file)

In [14]:
play("My name is nurse. How may I help you?",0)

HTTPError: HTTP Error 401: Unauthorized reqest.

In [11]:
#Configure microphone calibration
# Perform a quick test
r = sr.Recognizer()
with sr.Microphone() as source:
    #print("Please wait. Calibrating microphone...")
    # listen for 1 second and create the ambient noise energy level
    r.adjust_for_ambient_noise(source, duration=1)
    print("Say something!")
    audio = r.listen(source,phrase_time_limit=5)
    response = r.recognize_google(audio)
    print("I think you said: " + response)

Say something!
I think you said: hello


In [12]:
def printOut(msg, speech_out = False):
    print("👩 "+" >> " + msg +"\n")
    if(speech_out):
        play(msg)

In [28]:
responseslist = []
def init_nurse_bot(speech_in=False,speech_out=False):
    print("[INFO] The nurse is ready to listen. Please start a dialog... (Type 'stop' to quit)\n")
    errormsg = "I'm sorry, I didn't get it. Could you repeat please?"
    with sr.Microphone() as source:
        r.adjust_for_ambient_noise(source, duration=1)
        while True:
            # >>>>>>>>>>> Input sentence
            if(speech_in):
                flag = False
                cont = 0
                while(not flag):
                    try:
                        print("Listening... >> ", end='')
                        audio = r.listen(source,phrase_time_limit=3)
                        a = r.recognize_google(audio)
                        #parsetxt(a)
                        print(a, end='\n')
                        flag = True
                    except:
                        cont += 1
                        printOut(errormsg, False)
                        if(cont == 3):
                            print("\n[TERMINATED]", end='')
                            return
            else:
                print(">> ", end='')
                a = input()
            if a == 'stop':
                print("\n[TERMINATED]", end='')
                return
            responses = agent.handle_message(a)
            responseslist.append(responses)
            if(len(responses) == 0):
                print("Action returned 0 elements. Maybe the action listener is not active. Activate listener:\n python -m rasa_core_sdk.endpoint --actions actions")
            for response in responses:
                printOut(response["text"], speech_out)
                
                stopPhrases = [ "I'll be here if you need me.","Let me know if you need anything.","I'm at your service.","Call me if you need anything"]
                if(response["text"] in stopPhrases):
                    print("\n[TERMINATED]", end='')
                    return

In [ ]:
#Activate action listener in an external console!: 
# python -m rasa_core_sdk.endpoint --actions actions

In [30]:
init_nurse_bot(True,True)

[INFO] The nurse is ready to listen. Please start a dialog... (Type 'stop' to quit)

Listening... >> hello
👩  >> Hello! You can call me Nurse. How can I help you?

Listening... >> bring me my phone
👩  >> Sure! Here it is.

Listening... >> how to get to the bathroom
👩  >> Sure, let's go.

👩  >> Do you need anything else?

Listening... >> 👩  >> I'm sorry, I didn't get it. Could you repeat please?

Listening... >> 👩  >> I'm sorry, I didn't get it. Could you repeat please?

Listening... >> call an ambulance
👩  >> It's an emergency! I am getting the closest ambulance for you now. Hold there!

Listening... >> thank you
👩  >> My pleasure. 

Listening... >> 👩  >> I'm sorry, I didn't get it. Could you repeat please?

Listening... >> 👩  >> I'm sorry, I didn't get it. Could you repeat please?

Listening... >> 👩  >> I'm sorry, I didn't get it. Could you repeat please?


[TERMINATED]

In [ ]:
#!python -m rasa_core.run -d models/dialogue -u models/nursebot/interpreter --debug --endpoints endpoints.yml

In [17]:
responseslist

[[{'recipient_id': 'default',
   'text': 'Greetings. You can call me Nurse. How may I help you?'}],
 [{'recipient_id': 'default', 'text': 'Absolutely. Here it is.'}],
 [{'recipient_id': 'default', 'text': 'Sure, let me help you.'},
  {'recipient_id': 'default',
   'text': 'Is there something else I can do for you?'}]]